In [ ]:
import numpy as np
import pywt
import cv2
import os
from PIL import Image
from scipy.fftpack import dct
from scipy.fftpack import idct
import matplotlib.pyplot as plt

### Global Variables

In [ ]:
watermark = 'wm_rosace.jpg' #Name of the watermark image, must be in the "files" folder
video = 'oppenheimer_trailer.mp4' #Name of the video, must be in the "files" folder

file_folder = r'./files/' #Folder where the video and the watermark are stored
output_folder = r'./output/' #Folder where the output video will be stored

#Watermarking Parameters
stride = 4
block_ind = 2
wm_size = 256
coeff = 0.1

#Video Parameters
first_frame_number = -1
last_frame_number = 600

### Functions

In [ ]:
def convert_image(image_name, size): #Converts the watermark to the proper file format
    img = Image.open( file_folder + image_name)

    img = img.resize((size, size), 1)
    img = img.convert('L')
    
 
    image_array = np.array(img.getdata(), dtype=np.float).reshape((size, size))
       

    return image_array

In [ ]:
def convert_frame(image, size): #Converts the frame to the proper file format
    img = Image.fromarray(image)
    img = img.resize((size, size), 1)
    img = img.convert('L')

 
    image_array = np.array(img.getdata(), dtype=np.float).reshape((size, size))
           

    return image_array

In [ ]:
def process_coefficients(imArray, model, level): #Decomposes the frame into wavelet coefficients
    coeffs=pywt.wavedec2(data = imArray, wavelet = model, level = level)
    # print coeffs[0].__len__()
    coeffs_H=list(coeffs) 
   
    return coeffs_H


In [ ]:
def embed_mod2(coeff_image, coeff_watermark, offset=0): 
    for i in xrange(coeff_watermark.__len__()):
        for j in xrange(coeff_watermark[i].__len__()):
            coeff_image[i*2+offset][j*2+offset] = coeff_watermark[i][j]

    return coeff_image

In [ ]:
def embed_mod4(coeff_image, coeff_watermark):
    for i in xrange(coeff_watermark.__len__()):
        for j in xrange(coeff_watermark[i].__len__()):
            coeff_image[i*4][j*4] = coeff_watermark[i][j]

    return coeff_image

In [ ]:
def embed_watermark(watermark_array, orig_image): #Embeds the watermark into the frame as a flattened array
    watermark_array_size = watermark_array[0].__len__()
    watermark_flat = watermark_array.ravel()
    ind = 0
    
    for x in range (0, orig_image.__len__(), stride):
        for y in range (0, orig_image.__len__(), stride):
            if ind < watermark_flat.__len__():
                subdct = orig_image[x:x+stride, y:y+stride]
                subdct[block_ind][block_ind] = watermark_flat[ind] * coeff
                orig_image[x:x+stride, y:y+stride] = subdct
                ind += 1 

    return orig_image, watermark_flat

In [ ]:
def apply_dct(image_array): #Applies the DCT to the frame
    size = image_array[0].__len__()
    all_subdct = np.empty((size, size))
    for i in range (0, size, stride):
        for j in range (0, size, stride):
            subpixels = image_array[i:i+stride, j:j+stride]
            subdct = dct(dct(subpixels.T, norm="ortho").T, norm="ortho")
            all_subdct[i:i+stride, j:j+stride] = subdct

    return all_subdct

In [ ]:
def inverse_dct(all_subdct): #Applies the inverse DCT to the frame
    size = all_subdct[0].__len__()
    all_subidct = np.empty((size, size))
    for i in range (0, size, stride):
        for j in range (0, size, stride):
            subidct = idct(idct(all_subdct[i:i+stride, j:j+stride].T, norm="ortho").T, norm="ortho")
            all_subidct[i:i+stride, j:j+stride] = subidct

    return all_subidct

In [ ]:
def get_watermark(dct_watermarked_coeff, watermark_size): #Extracts the flat watermark from the frame
    
    subwatermarks = []

    for x in range (0, dct_watermarked_coeff.__len__(), stride):
        for y in range (0, dct_watermarked_coeff.__len__(), stride):
            coeff_slice = dct_watermarked_coeff[x:x+stride, y:y+stride]
            subwatermarks.append(coeff_slice[block_ind][block_ind] * 1/(2*coeff))

    watermark = np.array(subwatermarks).reshape((watermark_size, watermark_size))

    return watermark, subwatermarks

In [ ]:
def recover_watermark(image_array, model='haar', level = 1): #Recovers the watermark from the frame as a 2D image


    coeffs_watermarked_image = process_coefficients(image_array, model, level=level)
    dct_watermarked_coeff = apply_dct(coeffs_watermarked_image[0])
    
    watermark_array, flat_recovered = get_watermark(dct_watermarked_coeff, wm_size)

    watermark_array =  np.uint8(watermark_array)

#Save result
    img = Image.fromarray(watermark_array)
    return img, flat_recovered

In [ ]:
def save_watermark(frame_number, model='haar', level = 1): #Saves the recovered watermark as a 2D image
    image = Image.open(output_folder + r'frames/frame_{}.png'.format(frame_number))
    image = np.array(image)
    image = np.array(image, dtype=np.float)
    print(type(image))

    recovered, flat_recovered = recover_watermark(image_array = image, model=model, level = level)

    recovered.save(output_folder + r'recovered_watermark/recovered_watermark_{}.png'.format(frame_number))
    return recovered, flat_recovered


In [ ]:
def dist(l1, l2): #Calculates the distance between two flattened arrays
    return (sum((a - b) ** 2 for a, b in zip(l1, l2)) ** 0.5) / len(l1)

In [ ]:
def w2d(img): #Main function, embeds the watermark into the video, saves the output frames, and recovers the watermark
    model = 'haar'
    level = 1
    image_array = convert_frame(img, 2048)
    watermark_array = convert_image(watermark, wm_size)

    plt.imshow(watermark_array, cmap='gray')

    coeffs_image = process_coefficients(image_array, model, level=level)
    dct_array = apply_dct(coeffs_image[0])
    dct_array, watermark_original = embed_watermark(watermark_array, dct_array)
    coeffs_image[0] = inverse_dct(dct_array)
  

# reconstruction
    image_array_H=pywt.waverec2(coeffs_image, model)

    
    
# recover images
    return image_array_H, watermark_original

In [ ]:
cap = cv2.VideoCapture(file_folder + 'oppenheimer_trailer.mp4')


sequence = [] #List of watermark frames

assert cap.isOpened()

frame_number = first_frame_number
total_frames = last_frame_number

while True:

    frame_number += 1

    print("Watermarking frame n°{}".format(frame_number))

    cap.set(1, frame_number)
    ret, frame = cap.read()

    if not ret or frame_number >= last_frame_number:
        break

    # Modify the pixels of the frame here
    # For example, convert the frame to grayscale
    
    frame, w_original = w2d(frame) #Embeds the watermark into the frame, returns the watermarked frame and the flattened watermark
    
    cv2.imwrite(output_folder + 'frames/frame_{}.png'.format(frame_number), frame) #Saves the watermarked image

    # Write the modified frame to a new video file
    w_recovered, w_flat = save_watermark(frame_number) #Recovers the watermark from the watermarked frame, returns the recovered watermark and the flattened watermark

    sequence.append(np.array(w_recovered)) #Adds the recovered watermark to the list of watermarks

    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
def average_frames(sequence): #Computes the average frame for a sequence of frames
    average = np.zeros(sequence[0].shape)
    for frame in sequence:
        print(frame)
        average += frame
    average /= len(sequence)
    return average

avg = average_frames(sequence) #Computes the average watermark

plt.imshow(avg, cmap='gray')
plt.savefig(output_folder + 'average_wm.png')